In [ ]:
!pip install torch torchvision python-telegram-bot nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
#Conecta o ambiente ao seu google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import io
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes
import nest_asyncio
import asyncio

In [ ]:
#Permitir reentrância no loop de eventos
nest_asyncio.apply()

In [ ]:
#Função para carregar o modelo AlexNet
def carrega_modelo(caminho_do_modelo):
    modelo = models.alexnet(pretrained=True)
    num_ftrs = modelo.classifier[6].in_features
    modelo.classifier[6] = nn.Linear(num_ftrs, 2)
    modelo.load_state_dict(torch.load(caminho_do_modelo, map_location=torch.device('cpu')))
    modelo.eval()
    return modelo

In [ ]:
#Caminho para o modelo treinado
caminho_do_modelo = '/content/drive/My Drive/Best_model.pth'
modelo = carrega_modelo(caminho_do_modelo)

#Transformações de imagem
transforma = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

class_names = ['Arvore', 'Carro']

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:07<00:00, 34.4MB/s]


In [ ]:
#Função para classificar a imagem
def classifica_imagem(imagem):
    imagem = transforma(imagem).unsqueeze(0)
    with torch.no_grad():
        outputs = modelo(imagem)
        _, preds = torch.max(outputs, 1)
        class_idx = preds.item()
        class_name = class_names[class_idx]
    return class_name

#Função para responder ao comando /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text('Olá! Envie-me uma foto de um carro ou uma árvore para que eu possa classificá-la.')

#Função para responder a fotos enviadas
async def handle_photo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    photo_file = await update.message.photo[-1].get_file()
    photo_bytes = await photo_file.download_as_bytearray()
    imagem = Image.open(io.BytesIO(photo_bytes))
    class_name = classifica_imagem(imagem)
    await update.message.reply_text(f'Eu acho que isso é um(a) {class_name}')


In [ ]:
#Função para configurar o bot
async def main():
    application = ApplicationBuilder().token("7125213193:AAEDhMZns3fY1eD0r39-oKK70NmBkKkU31Y").build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.PHOTO, handle_photo))

    await application.run_polling()

link para o bot no linkedin: t.me/ClassificadorBot

In [ ]:
#Executa o bot
await main()

RuntimeError: Cannot close a running event loop